In [1]:
from progressivis_nb_widgets import *

In [2]:
from progressivis import Scheduler, Every#, log_level
from progressivis.io import CSVLoader
from progressivis.stats.ppca import PPCA
from progressivis.core import aio
from progressivis.stats.blobs_table import BlobsTable
from progressivis.vis import MCScatterPlot
from progressivis.utils.psdict import PsDict
from progressivis.core.utils import gather_and_run
import numpy as np
import pandas as pd

async def moderator(a, b):
    await aio.sleep(0.05)

try:
    s = scheduler
except NameError:
    s = Scheduler()

n_samples = 10000000
n_components = 2
rtol = 0.01
centers = [(0.1, 0.3, 0.5), (0.7, 0.5, 3.3), (-0.4, -0.3, -11.1)]
data = BlobsTable(columns=['_0', '_1', '_2'],  centers=centers,
                          cluster_std=0.2, rows=n_samples, scheduler=s)
ppca = PPCA(scheduler=s)
ppca.input.table = data.output.table
ppca.params.n_components = n_components
ppca.create_dependent_modules(rtol=rtol)
cnames = ['samples', 'prev_samples']
classes = []
for cn in cnames:
    classes.append({'name': cn, 'x_column': '_pc0',
                    'y_column': '_pc1', 'sample': ppca.reduced, 'sample_slot': cn,
                    'input_module': ppca.reduced, 'input_slot': 'table'})
sp = MCScatterPlot(scheduler=s, classes=classes)
sp.create_dependent_modules()
for cn in cnames:
    sp[cn].min_value._table = PsDict({'_pc0': -np.inf, '_pc1': -np.inf})
    sp[cn].max_value._table = PsDict({'_pc0': np.inf, '_pc1': np.inf})
sp_widget = Scatterplot()
widget_coros = sp_widget.link_module(sp)
gather_and_run(s.start(tick_proc=moderator), *widget_coros)

In [3]:
display(sp_widget)

Scatterplot(to_hide=[])